### Codes to read directly from Market Index website
##### Put into dataframe
###### Do some data cleaning
###### Export the clean data to CSV.

In [75]:
# Dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

Find all the data pertaining to ASX tickers listed in this website.

This is how the website for "BHP" and the bhp will be changed to cba for "VBA" ticker etc.

https://www.marketindex.com.au/asx/bhp

#### Codes to retrieve all the ASX data direct from Market Index website and put into dataframe

In [57]:
# Use the read_csv function to read the CSV file
df = pd.read_csv("Resources/ASX_Top10_Industry.csv")

df.head(10)

,ticker,company_name,industry_gp,market_cap
0,ARB,ARB CORPORATION LIMITED.,Automobiles & Components,2.455925e+09
1,GUD,G.U.D. HOLDINGS LIMITED,Automobiles & Components,1.534343e+09
2,PWH,PWR HOLDINGS LIMITED,Automobiles & Components,1.000822e+09
3,SFC,SCHAFFER CORPORATION LIMITED,Automobiles & Components,2.272901e+08
4,AHL,ADRAD HOLDINGS LIMITED,Automobiles & Components,7.275023e+07
5,VMT,VMOTO LIMITED,Automobiles & Components,4.353604e+07
6,RPM,RPM AUTOMOTIVE GROUP LIMITED,Automobiles & Components,1.573357e+07
7,ABV,ADVANCED BRAKING TECHNOLOGY LTD,Automobiles & Components,1.555814e+07
8,DDT,DATADOT TECHNOLOGY LIMITED,Automobiles & Components,4.843811e+06
9,SIX,SPRINTEX LIMITED,Automobiles & Components,4.745972e+06


## List of Top10 per Industry to extract the data

In [60]:
# Extract the "Code" column as a list
tickers = df['ticker'].tolist()

# make copies
copied_tickers = tickers.copy()
print(copied_tickers)

['ARB', 'GUD', 'PWH', 'SFC', 'AHL', 'VMT', 'RPM', 'ABV', 'DDT', 'SIX', 'CBA', 'NAB', 'ANZ', 'WBC', 'BEN', 'VUK', 'BOQ', 'BFL', 'HGH', 'JDO', 'REH', 'SVW', 'WOR', 'IFT', 'FBU', 'RWC', 'VNT', 'JLG', 'MND', 'RDX', 'SLB', 'TRE', 'OXT', 'S3N', 'HRE', 'KLI', 'DY6', 'BXB', 'CPU', 'ALQ', 'CWY', 'DOW', 'APM', 'IPH', 'MAD', 'MMS', 'SIQ', 'WES', 'JBH', 'HVN', 'PMV', 'APE', 'SUL', 'LOV', 'BAP', 'AX1', 'BGP', 'BRG', 'AKP', 'FWD', 'WAT', 'AVJ', 'GLB', 'TWD', 'SHM', 'SIO', 'GAP', 'ALL', 'LNW', 'TLC', 'IEL', 'DMP', 'FLT', 'CTD', 'WEB', 'TAH', 'IVC', 'WOW', 'COL', 'EDV', 'MTS', 'GNC', 'MMM', 'WDS', 'STO', 'ALD', 'YAL', 'WHC', 'NXG', 'NHC', 'VEA', 'BPT', 'PDN', 'GMG', 'SCG', 'URW', 'SGP', 'VCX', 'MGR', 'GPT', 'DXS', 'CHC', 'NSR', 'MQG', 'SQ2', 'SOL', 'ASX', 'JHG', 'CGF', 'GQG', 'NWL', 'AMP', 'HUB', 'TWE', 'A2M', 'CGC', 'ING', 'ELD', 'BGA', 'AAC', 'RIC', 'CBO', 'SHV', 'RMD', 'COH', 'SHL', 'FPH', 'RHC', 'PME', 'EBO', 'ANN', 'SNZ', 'NAN', 'PTL', 'MCP', 'BIO', 'BXN', 'S66', 'HPC', 'EXL', 'SKN', 'CCO', 'HCT'

## Create function to extract the data one ticker at a time due to anti-bot website feature

In [49]:
def scrape_stock_data(ticker):
    # Set up Splinter
    browser = Browser('chrome')

    # Define the URL of the website you want to scrape for the given ticker
    url = f"https://www.marketindex.com.au/asx/{ticker}"

    # Use Splinter to open the URL
    browser.visit(url)

    # Get the HTML content from the page
    html_content = browser.html

    # Create a BeautifulSoup object to parse the HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Define a function to extract data from a table
    def extract_data_from_table(table):
        data = {}
        rows = table.find_all('tr')
        for row in rows:
            columns = row.find_all('td')
            if len(columns) == 2:
                attribute = columns[0].text.strip()
                value = columns[1].text.strip()
                data[attribute] = value
        return data

    # Find and extract data from different tables
    share_price_table = soup.find('h3', string='Share Price Activity (ASX)').find_next('table', class_='mi-table')
    performance_table = soup.find('h3', string='Performance').find_next('table', class_='mi-table')
    size_table = soup.find('h3', string='Size').find_next('table', class_='mi-table')
    fundamentals_table = soup.find('h3', string='Key Fundamentals').find_next('table', class_='mi-table')
    broker_consensus_table = soup.find('h3', string='Broker Consensus').find_next('table', class_='mi-table')

    # Extract data from the tables
    share_price_data = extract_data_from_table(share_price_table)
    performance_data = extract_data_from_table(performance_table)
    size_data = extract_data_from_table(size_table)
    fundamentals_data = extract_data_from_table(fundamentals_table)
    broker_consensus_data = extract_data_from_table(broker_consensus_table)

    # Combine all the extracted data into one dictionary
    all_data = {**share_price_data, **performance_data, **size_data, **fundamentals_data, **broker_consensus_data}

    # Create a DataFrame
    df = pd.DataFrame(list(all_data.items()), columns=['Attribute', 'Value'])

    # Transpose the DataFrame
    df_transposed = df.T

    # Set the "Attribute" column as the index
    df_transposed.columns = df_transposed.loc['Attribute']

    # Remove the duplicated "Attribute" row
    df_transposed = df_transposed[1:]

    # Reset the index and add a "Ticker" column with the given ticker
    df_transposed = df_transposed.reset_index()
    df_transposed = df_transposed.rename(columns={'Attribute': 'Ticker'})
    df_transposed['Ticker'] = ticker

    # Close the browser
    browser.quit()

    return df_transposed

# Example usage:
ticker = 'BHP'  # Replace with the desired ticker
bhp_df = scrape_stock_data(ticker)
bhp_df

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$45.50,0.12\n(0.26%),$45.50\n-\n$45.52,"6,766,318","7,345,298","$308,576,546",$45.80,$45.43 - $45.88,$45.61,...,"5,068,739,787",Basic Materials,FMG / RIO / NCM,$2.547,$2.6143,$8.184,11 Buy\n· 9 Hold\n· 3 Sell,Buy,01/10/23,BHP


## This works better but some issue with certain ticker having the ? symbol next to the text.

In [87]:
def scrape_stock_data2(ticker):
    # Set up Splinter
    browser = Browser('chrome')

    # Define the URL of the website you want to scrape for the given ticker
    url = f"https://www.marketindex.com.au/asx/{ticker}"

    # Use Splinter to open the URL
    browser.visit(url)

    # Get the HTML content from the page
    html_content = browser.html
    
    #  Wait for website to load
    # website has anti bot protection.  Sometimes, the scarping doesn't work all at once.
    time.sleep(5)
    
    # Create a BeautifulSoup object to parse the HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Define a function to extract data from a table
    def extract_data_from_table(table):
        data = {}
        rows = table.find_all('tr')
        for row in rows:
            columns = row.find_all('td')
            if len(columns) == 2:
                attribute = columns[0].text.strip()
                value = columns[1].text.strip()
                data[attribute] = value
        return data

    # Find and extract data from different tables
    # Find and extract data from the "Share Price Activity (ASX)" table
    share_price_table = soup.find('h3', string='Share Price Activity (ASX)')
    if share_price_table:
        share_price_table = share_price_table.find_next('table', class_='mi-table')
        share_price_data = extract_data_from_table(share_price_table)
    else:
        share_price_data = {}  # or another default value or error handling

    # Find and extract data from the "Performance)" table
    performance_table = soup.find('h3', string='Performance')
    if performance_table:
        performance_table = performance_table.find_next('table', class_='mi-table')
        performance_data = extract_data_from_table(performance_table)
    else:
        performance_data = {}  # or another default value or error handling

    # Find and extract data from the "Size" table
    size_table = soup.find('h3', string='Size')
    if size_table:
        size_table = size_table.find_next('table', class_='mi-table')
        size_data = extract_data_from_table(size_table)
    else:
        size_data = {}  # or another default value or error handling

    # Find and extract data from the "Key Fundamentals" table
    fundamentals_table = soup.find('h3', string='Key Fundamentals')
    if fundamentals_table:
        fundamentals_table = fundamentals_table.find_next('table', class_='mi-table')
        fundamentals_data = extract_data_from_table(fundamentals_table)
    else:
        fundamentals_data = {}  # or another default value or error handling

    # Find and extract data from the "Broker Consensus" table
    broker_consensus_table = soup.find('h3', string='Broker Consensus')
    if broker_consensus_table:
        broker_consensus_table = broker_consensus_table.find_next('table', class_='mi-table')
        broker_consensus_data = extract_data_from_table(broker_consensus_table)
    else:
        broker_consensus_data = {}  # or another default value or error handling

    # Combine all the extracted data into one dictionary
    all_data = {**share_price_data, **performance_data, **size_data, **fundamentals_data, **broker_consensus_data}

    # Create a DataFrame
    df = pd.DataFrame(list(all_data.items()), columns=['Attribute', 'Value'])

    # Transpose the DataFrame
    df_transposed = df.T

    # Set the "Attribute" column as the index
    df_transposed.columns = df_transposed.loc['Attribute']

    # Remove the duplicated "Attribute" row
    df_transposed = df_transposed[1:]

    # Reset the index and add a "Ticker" column with the given ticker
    df_transposed = df_transposed.reset_index()
    df_transposed = df_transposed.rename(columns={'Attribute': 'Ticker'})
    df_transposed['Ticker'] = ticker

    # Close the browser
    browser.quit()

    return df_transposed

# Example usage:
ticker = 'NAB'  # Replace with the desired ticker
nab_df = scrape_stock_data2(ticker)
nab_df


Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$29.06,0.49\n(1.72%),$28.92\n-\n$29.10,"4,699,140","4,334,055","$136,415,820",$28.95,$28.78 - $29.13,$29.025,...,"3,128,948,927",Financial Services,ANZ / WBC / MQG,$2.268,$1.61,$17.372,9 Buy\n· 5 Hold\n· 4 Sell,Hold,01/10/23,NAB


In [51]:
def scrape_stock_data3(ticker):
    # Set up Splinter
    browser = Browser('chrome')

    # Define the URL of the website you want to scrape for the given ticker
    url = f"https://www.marketindex.com.au/asx/{ticker}"

    # Use Splinter to open the URL
    browser.visit(url)

    # Get the HTML content from the page
    html_content = browser.html
    
    # Wait for the website to load
    time.sleep(5)

    # Create a BeautifulSoup object to parse the HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Define a function to extract data from a table
    def extract_data_from_table(table):
        data = {}
        rows = table.find_all('tr')
        for row in rows:
            columns = row.find_all('td')
            if len(columns) == 2:
                attribute = columns[0].text.strip()
                value = columns[1].text.strip()
                data[attribute] = value
        return data

    data = {}  # Initialize an empty dictionary to store data

    # Define the list of table headers to extract
    table_headers = ['Share Price Activity (ASX)', 'Performance', 'Size', 'Key Fundamentals', 'Broker Consensus']

    # Extract data from each table
    for header in table_headers:
        table = soup.find('h3', string=header)
        if table:
            table = table.find_next('table', class_='mi-table')
            table_data = extract_data_from_table(table)
        else:
            table_data = {}  # Default to an empty dictionary if the table is not found
        data.update(table_data)

    # Close the browser
    browser.quit()

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(list(data.items()), columns=['Attribute', 'Value'])

    # Transpose the DataFrame
    df = df.T

    # Set the "Attribute" column as the index
    df.columns = df.loc['Attribute']

    # Remove the duplicated "Attribute" row
    df = df[1:]

    # Reset the index and add a "Ticker" column with the given ticker
    df = df.reset_index()
    df = df.rename(columns={'Attribute': 'Ticker'})
    df['Ticker'] = ticker

    return df

# Example usage:
ticker = 'BHP'  # Replace with the desired ticker
bhp_df = scrape_stock_data3(ticker)
bhp_df


Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$45.50,0.12\n(0.26%),$45.50\n-\n$45.52,"6,766,318","7,345,298","$308,576,546",$45.80,$45.43 - $45.88,$45.61,...,"5,068,739,787",Basic Materials,FMG / RIO / NCM,$2.547,$2.6143,$8.184,11 Buy\n· 9 Hold\n· 3 Sell,Buy,01/10/23,BHP


In [45]:
ticker = 'SLB'  # Replace with the desired ticker
bhp_df = scrape_stock_data(ticker)
bhp_df

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Market Cap\n\n\n\n\n\n\n\n\n\nExcludes mandatory escrowed shares and foreign shares.,ASX Rank,Sector Rank,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Ticker
0,Value,$0.285,0.025\n(9.62%),$0.27\n-\n$0.285,"110,064","191,029","$30,873",$0.265,$0.265 - $0.29,$0.2805,...,"$14,793,759","1,687 of 2,408",581 of 978,"51,907,925",Basic Materials,ADV / ASE / MRZ,-$0.044,$0.00,$0.123,SLB


In [34]:
print(tickers)

['ARB', 'GUD', 'PWH', 'SFC', 'AHL', 'VMT', 'RPM', 'ABV', 'DDT', 'SIX', 'CBA', 'NAB', 'ANZ', 'WBC', 'BEN', 'VUK', 'BOQ', 'BFL', 'HGH', 'JDO', 'REH', 'SVW', 'WOR', 'IFT', 'FBU', 'RWC', 'VNT', 'JLG', 'MND', 'RDX', 'SLB', 'TRE', 'OXT', 'S3N', 'HRE', 'KLI', 'DY6', 'BXB', 'CPU', 'ALQ', 'CWY', 'DOW', 'APM', 'IPH', 'MAD', 'MMS', 'SIQ', 'WES', 'JBH', 'HVN', 'PMV', 'APE', 'SUL', 'LOV', 'BAP', 'AX1', 'BGP', 'BRG', 'AKP', 'FWD', 'WAT', 'AVJ', 'GLB', 'TWD', 'SHM', 'SIO', 'GAP', 'ALL', 'LNW', 'TLC', 'IEL', 'DMP', 'FLT', 'CTD', 'WEB', 'TAH', 'IVC', 'WOW', 'COL', 'EDV', 'MTS', 'GNC', 'MMM', 'WDS', 'STO', 'ALD', 'YAL', 'WHC', 'NXG', 'NHC', 'VEA', 'BPT', 'PDN', 'GMG', 'SCG', 'URW', 'SGP', 'VCX', 'MGR', 'GPT', 'DXS', 'CHC', 'NSR', 'MQG', 'SQ2', 'SOL', 'ASX', 'JHG', 'CGF', 'GQG', 'NWL', 'AMP', 'HUB', 'TWE', 'A2M', 'CGC', 'ING', 'ELD', 'BGA', 'AAC', 'RIC', 'CBO', 'SHV', 'RMD', 'COH', 'SHL', 'FPH', 'RHC', 'PME', 'EBO', 'ANN', 'SNZ', 'NAN', 'PTL', 'MCP', 'BIO', 'BXN', 'S66', 'HPC', 'EXL', 'SKN', 'CCO', 'HCT'

In [77]:
# Please note thate scarping the website for 259 tickers took mroe than 1 hour and some of the ticker
# were not scraped successfully.  Those have to be extracted individualy and added to the final dataframe.

# To test the code for small amunt of tickers, 
# modify the for ticker in tickers to 
# for ticker in tickers[0:5] to run for the 1st 5 tickers

# Initialize an empty list to store DataFrames
dfs = []

# Iterate through tickers
for ticker in tickers:
    df = scrape_stock_data2(ticker)
    dfs.append(df)

# Concatenate the list of DataFrames into a single DataFrame
final_df = pd.concat(dfs, ignore_index=True)

# Print the final DataFrame
final_df

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$31.26,1.02\n(3.37%),$29.88\n-\n$31.35,"160,245","245,483","$4,999,714",$31.07,$30.56 - $31.45,$31.20,...,"82,220,441",Consumer Cyclical,CTD / WEB / SUL,$1.079,$0.62,$0.60,6 Buy\n· 5 Hold\n· 2 Sell,Hold,01/10/23,ARB
1,Value,$11.08,0.17\n(1.56%),$10.77\n-\n$11.35,"366,353","441,797","$4,045,735",$10.95,$10.725 - $11.10,$11.045,...,"140,894,696",Consumer Cyclical,SGR / LNW / SKC,$0.693,$0.39,$0.526,11 Buy\n· 2 Hold\n· 0 Sell,Strong Buy,01/10/23,GUD
2,Value,$10.12,0.07\n(0.70%),$9.96\n-\n$10.92,"37,398","127,350","$378,557",$10.16,$10.06 - $10.22,$10.12,...,"100,484,131",Consumer Cyclical,BGP / CTT / AX1,$0.217,$0.125,$0.722,7 Buy\n· 4 Hold\n· 1 Sell,Buy,01/10/23,PWH
3,Value,$16.10,0.65\n(-3.88%),$16.15\n-\n$17.10,"5,570",758,"$90,352",$16.75,$16.01 - $16.75,$16.22,...,"13,569,557",Consumer Cyclical,NDO / ALG / BST,$1.811,$0.90,$3.324,NaN,NaN,NaN,SFC
4,Value,$0.89,0.00\n(0.00),$0.845\n-\n$0.90,0,"11,146",$0,‐,‐ - ‐,‐,...,"81,285,172",Consumer Cyclical,ISU / KED / CCX,$0.084,$0.0233,$0.986,NaN,NaN,NaN,AHL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,Value,$7.29,0.04\n(0.55%),$7.08\n-\n$7.70,"3,602","9,956","$26,207",$7.27,$7.25 - $7.30,$7.275,...,"254,546,234",Utilities,GNE / IFN / TLT,$0.163,$0.3167,$3.014,7 Buy\n· 2 Hold\n· 0 Sell,Strong Buy,01/10/23,CEN
255,Value,$2.17,0.02\n(0.93%),$2.11\n-\n$2.20,"18,082","20,899","$39,339",$2.15,$2.15 - $2.20,$2.175,...,"1,074,640,281",Utilities,CEN / TLT / IFN,$0.185,$0.1642,$1.966,5 Buy\n· 3 Hold\n· 1 Sell,Buy,01/10/23,GNE
256,Value,$1.635,0.015\n(0.93%),$1.635\n-\n$1.67,"38,498","79,426","$63,254",$1.64,$1.635 - $1.67,$1.643,...,"152,720,578",Utilities,GNX / LGI / EWC,$0.00,$0.069,$0.00,NaN,NaN,NaN,D2O
257,Value,$0.16,0.01\n(6.67%),$0.155\n-\n$0.16,"1,750,005","372,851","$272,608",$0.155,$0.1525 - $0.16,$0.156,...,"1,385,177,140",Utilities,D2O / LGI / EWC,-$0.001,$0.00,$0.143,5 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,GNX


In [84]:
# Find rows with an empty "Last (Price)" column
empty_last_price_rows = final_df[pd.isna(final_df['Last (Price)']) | (final_df['Last (Price)'] == '')]

# Print the rows with an empty "Last (Price)" column
empty_last_price_rows

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
16,Value,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BOQ
55,Value,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AX1
62,Value,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GLB
68,Value,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LNW
83,Value,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WDS
84,Value,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STO
99,Value,,0.00\n(0.00),-,,,,,-,,...,,Real Estate,DXS / MGR / VCX,-$0.033,$0.248,$5.978,8 Buy\n· 3 Hold\n· 4 Sell,Hold,01/10/23,GPT
102,Value,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NSR
124,Value,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COH
169,Value,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DHG


In [86]:
tickers2 = empty_last_price_rows['Ticker'].tolist()
print(tickers2)

['BOQ', 'AX1', 'GLB', 'LNW', 'WDS', 'STO', 'GPT', 'NSR', 'COH', 'DHG', 'IMU', 'XRO', 'AD8', 'FRW']


In [88]:
#  Redo those Tickers that weren't successfulluy scraped.
# Initialize an empty list to store DataFrames
dfs = []

# Iterate through tickers
for ticker in tickers2:
    df = scrape_stock_data2(ticker)
    dfs.append(df)

# Concatenate the list of DataFrames into a single DataFrame
final_df2 = pd.concat(dfs, ignore_index=True)

# Print the final DataFrame
final_df2

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$5.45,0.19\n(3.61%),$5.42\n-\n$5.45,"2,691,645","3,507,998","$14,564,520",$5.30,$5.30 - $5.45,$5.41,...,"657,217,431",Financial Services,NHF / VTS / NDQ,$0.183,$0.44,$7.734,2 Buy\n· 6 Hold\n· 12 Sell,Sell,01/10/23,BOQ
1,Value,$1.955,0.02\n(1.03%),$1.945\n-\n$1.96,"1,673,098","825,084","$3,260,834",$1.935,$1.935 - $1.975,$1.953,...,"563,053,196",Consumer Cyclical,CKF / CTT / PWH,$0.156,$0.175,$0.105,7 Buy\n· 5 Hold\n· 2 Sell,Buy,01/10/23,AX1
2,Value,$3.10,0.05\n(1.64%),$2.85\n-\n$3.09,526,"1,341","$1,631",$3.10,$3.10 - $3.10,$3.10,...,"41,463,818",Consumer Cyclical,AVJ / STP / VVA,$0.038,$0.07,$1.694,NaN,NaN,NaN,GLB
3,Value,$117.93,4.14\n(3.64%),‐\n-\n‐,"127,849","91,994","$15,083,912",$117.78,$116.24 - $117.93,$117.56,...,"14,626,044",Consumer Cyclical,IVC / SGR / TAH,$0.46,$0.00,-$31.426,10 Buy\n· 4 Hold\n· 0 Sell,Strong Buy,01/10/23,LNW
4,Value,$34.01,0.16\n(0.47%),$34.00\n-\n$34.05,"4,262,254","5,111,264","$145,184,300",$34.10,$33.81 - $34.28,$34.01,...,"1,898,749,771",Energy,WPL / STO / OSH,$3.734,$3.3974,$16.707,8 Buy\n· 7 Hold\n· 4 Sell,Hold,01/10/23,WDS
5,Value,$7.42,0.09\n(-1.20%),$7.39\n-\n$7.45,"10,850,167","8,129,617","$80,681,152",$7.59,$7.36 - $7.60,$7.435,...,"3,247,772,961",Energy,WPL / OSH / ALD,$0.523,$0.3576,$4.204,15 Buy\n· 4 Hold\n· 0 Sell,Strong Buy,01/10/23,STO
6,Value,$3.89,0.08\n(2.10%),$3.86\n-\n$3.90,"7,198,814","4,961,637","$27,992,625",$3.92,$3.85 - $3.93,$3.89,...,"1,915,577,430",Real Estate,DXS / MGR / VCX,-$0.033,$0.248,$5.978,8 Buy\n· 3 Hold\n· 4 Sell,Hold,01/10/23,GPT
7,Value,$2.06,0.03\n(1.48%),$2.04\n-\n$2.08,"4,216,672","2,726,955","$8,689,761",$2.07,$2.04 - $2.08,$2.06,...,"1,361,129,474",Real Estate,SCP / RGN / CLW,$0.258,$0.11,$0.166,6 Buy\n· 5 Hold\n· 2 Sell,Hold,01/10/23,NSR
8,Value,$247.58,1.65\n(0.67%),$245.70\n-\n$248.52,"104,162","140,143","$25,796,002",$247.27,$245.68 - $249.17,$247.655,...,"65,533,900",Healthcare,SHL / RMD / FPH,$4.561,$3.30,$0.097,3 Buy\n· 12 Hold\n· 9 Sell,Sell,01/10/23,COH
9,Value,$3.60,0.00\n(0.00),$3.58\n-\n$3.62,"1,181,415","723,777","$4,299,739",$3.63,$3.60 - $3.71,$3.64,...,"631,657,153",Communication Services,EVT / CNU / NEC,$0.058,$0.06,-$0.449,1 Buy\n· 10 Hold\n· 6 Sell,Sell,01/10/23,DHG


In [92]:
# Drop rows where "Last (Price)" is NaN
final_df1 = final_df.dropna(subset=['Last (Price)'])

# Print the resulting DataFrame
final_df1

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$31.26,1.02\n(3.37%),$29.88\n-\n$31.35,"160,245","245,483","$4,999,714",$31.07,$30.56 - $31.45,$31.20,...,"82,220,441",Consumer Cyclical,CTD / WEB / SUL,$1.079,$0.62,$0.60,6 Buy\n· 5 Hold\n· 2 Sell,Hold,01/10/23,ARB
1,Value,$11.08,0.17\n(1.56%),$10.77\n-\n$11.35,"366,353","441,797","$4,045,735",$10.95,$10.725 - $11.10,$11.045,...,"140,894,696",Consumer Cyclical,SGR / LNW / SKC,$0.693,$0.39,$0.526,11 Buy\n· 2 Hold\n· 0 Sell,Strong Buy,01/10/23,GUD
2,Value,$10.12,0.07\n(0.70%),$9.96\n-\n$10.92,"37,398","127,350","$378,557",$10.16,$10.06 - $10.22,$10.12,...,"100,484,131",Consumer Cyclical,BGP / CTT / AX1,$0.217,$0.125,$0.722,7 Buy\n· 4 Hold\n· 1 Sell,Buy,01/10/23,PWH
3,Value,$16.10,0.65\n(-3.88%),$16.15\n-\n$17.10,"5,570",758,"$90,352",$16.75,$16.01 - $16.75,$16.22,...,"13,569,557",Consumer Cyclical,NDO / ALG / BST,$1.811,$0.90,$3.324,NaN,NaN,NaN,SFC
4,Value,$0.89,0.00\n(0.00),$0.845\n-\n$0.90,0,"11,146",$0,‐,‐ - ‐,‐,...,"81,285,172",Consumer Cyclical,ISU / KED / CCX,$0.084,$0.0233,$0.986,NaN,NaN,NaN,AHL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,Value,$7.29,0.04\n(0.55%),$7.08\n-\n$7.70,"3,602","9,956","$26,207",$7.27,$7.25 - $7.30,$7.275,...,"254,546,234",Utilities,GNE / IFN / TLT,$0.163,$0.3167,$3.014,7 Buy\n· 2 Hold\n· 0 Sell,Strong Buy,01/10/23,CEN
255,Value,$2.17,0.02\n(0.93%),$2.11\n-\n$2.20,"18,082","20,899","$39,339",$2.15,$2.15 - $2.20,$2.175,...,"1,074,640,281",Utilities,CEN / TLT / IFN,$0.185,$0.1642,$1.966,5 Buy\n· 3 Hold\n· 1 Sell,Buy,01/10/23,GNE
256,Value,$1.635,0.015\n(0.93%),$1.635\n-\n$1.67,"38,498","79,426","$63,254",$1.64,$1.635 - $1.67,$1.643,...,"152,720,578",Utilities,GNX / LGI / EWC,$0.00,$0.069,$0.00,NaN,NaN,NaN,D2O
257,Value,$0.16,0.01\n(6.67%),$0.155\n-\n$0.16,"1,750,005","372,851","$272,608",$0.155,$0.1525 - $0.16,$0.156,...,"1,385,177,140",Utilities,D2O / LGI / EWC,-$0.001,$0.00,$0.143,5 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,GNX


In [93]:
# Append df2 to df1
result_df = pd.concat([final_df1, final_df2], ignore_index=True)

# Print the resulting DataFrame
result_df

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$31.26,1.02\n(3.37%),$29.88\n-\n$31.35,"160,245","245,483","$4,999,714",$31.07,$30.56 - $31.45,$31.20,...,"82,220,441",Consumer Cyclical,CTD / WEB / SUL,$1.079,$0.62,$0.60,6 Buy\n· 5 Hold\n· 2 Sell,Hold,01/10/23,ARB
1,Value,$11.08,0.17\n(1.56%),$10.77\n-\n$11.35,"366,353","441,797","$4,045,735",$10.95,$10.725 - $11.10,$11.045,...,"140,894,696",Consumer Cyclical,SGR / LNW / SKC,$0.693,$0.39,$0.526,11 Buy\n· 2 Hold\n· 0 Sell,Strong Buy,01/10/23,GUD
2,Value,$10.12,0.07\n(0.70%),$9.96\n-\n$10.92,"37,398","127,350","$378,557",$10.16,$10.06 - $10.22,$10.12,...,"100,484,131",Consumer Cyclical,BGP / CTT / AX1,$0.217,$0.125,$0.722,7 Buy\n· 4 Hold\n· 1 Sell,Buy,01/10/23,PWH
3,Value,$16.10,0.65\n(-3.88%),$16.15\n-\n$17.10,"5,570",758,"$90,352",$16.75,$16.01 - $16.75,$16.22,...,"13,569,557",Consumer Cyclical,NDO / ALG / BST,$1.811,$0.90,$3.324,NaN,NaN,NaN,SFC
4,Value,$0.89,0.00\n(0.00),$0.845\n-\n$0.90,0,"11,146",$0,‐,‐ - ‐,‐,...,"81,285,172",Consumer Cyclical,ISU / KED / CCX,$0.084,$0.0233,$0.986,NaN,NaN,NaN,AHL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,Value,$3.60,0.00\n(0.00),$3.58\n-\n$3.62,"1,181,415","723,777","$4,299,739",$3.63,$3.60 - $3.71,$3.64,...,"631,657,153",Communication Services,EVT / CNU / NEC,$0.058,$0.06,-$0.449,1 Buy\n· 10 Hold\n· 6 Sell,Sell,01/10/23,DHG
256,Value,$0.048,0.005\n(11.63%),$0.047\n-\n$0.048,"31,216,017","23,908,889","$1,450,452",$0.043,$0.043 - $0.049,$0.0465,...,"7,164,974,859",Healthcare,AFP / IMM / MYX,-$0.006,$0.00,$0.00,5 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,IMU
257,Value,$112.29,1.54\n(1.39%),$111.95\n-\n$112.60,"326,421","381,557","$36,711,769",$113.50,$110.37 - $113.67,$112.515,...,"151,468,059",Technology,APT / WTC / NXT,-$0.711,$0.00,$0.556,15 Buy\n· 4 Hold\n· 2 Sell,Buy,01/10/23,XRO
258,Value,$13.22,0.42\n(-3.08%),$13.22\n-\n$13.49,"78,352","159,319","$1,045,137",$13.51,$13.20 - $13.63,$13.34,...,"83,108,836",Technology,DTL / HSN / NEA,-$0.035,$0.00,$0.671,8 Buy\n· 2 Hold\n· 1 Sell,Buy,01/10/23,AD8


In [78]:
final_df1_1st_success = final_df.copy()

In [31]:
final_df2 = final_df.copy()

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$29.06,0.49\n(1.72%),$28.92\n-\n$29.10,"4,699,140","4,334,055","$136,415,820",$28.95,$28.78 - $29.13,$29.025,...,"3,128,948,927",Financial Services,ANZ / WBC / MQG,$2.268,$1.61,$17.372,9 Buy\n· 5 Hold\n· 4 Sell,Hold,01/10/23,NAB
1,Value,$25.60,0.23\n(0.91%),$25.59\n-\n$25.63,"5,813,929","5,496,181","$148,785,898",$25.73,$25.56 - $25.75,$25.635,...,"3,005,286,886",Financial Services,WBC / MQG / NAB,$2.297,$1.55,$20.682,12 Buy\n· 4 Hold\n· 2 Sell,Buy,01/10/23,ANZ
2,Value,$21.50,0.33\n(1.56%),$21.44\n-\n$21.50,"7,177,724","6,695,917","$154,169,117",$21.46,$21.39 - $21.58,$21.485,...,"3,509,076,960",Financial Services,ANZ / MQG / NAB,$1.744,$1.34,$17.161,3 Buy\n· 6 Hold\n· 10 Sell,Sell,01/10/23,WBC
3,Value,$9.09,0.20\n(2.25%),$9.05\n-\n$9.10,"1,285,689","1,174,747","$11,645,116",$8.96,$8.93 - $9.10,$9.055,...,"567,747,187",Financial Services,SDF / STW / MLT,$0.792,$0.61,$8.851,7 Buy\n· 8 Hold\n· 2 Sell,Hold,01/10/23,BEN
4,Value,$3.01,0.13\n(4.51%),$2.98\n-\n$3.02,"10,515,183","1,510,611","$31,555,235",$2.96,$2.94 - $3.015,$2.985,...,"672,361,526",Financial Services,IAF / VDHG / PDL,$0.337,$0.1922,$4.532,13 Buy\n· 8 Hold\n· 3 Sell,Buy,01/10/23,VUK
5,Value,$5.45,0.19\n(3.61%),$5.42\n-\n$5.45,"2,691,645","3,507,998","$14,564,520",$5.30,$5.30 - $5.45,$5.41,...,"657,217,431",Financial Services,NHF / VTS / NDQ,$0.183,$0.44,$7.734,2 Buy\n· 6 Hold\n· 12 Sell,Sell,01/10/23,BOQ
6,Value,$5.31,0.21\n(4.12%),$5.10\n-\n$5.35,"22,533","7,803","$116,762",$5.10,$5.10 - $5.35,$5.18,...,"467,219,979",Financial Services,VGAD / MGF / ETHI,$2.38,$0.6258,$7.958,NaN,NaN,NaN,BFL
7,Value,$1.415,0.00\n(0.00),$1.385\n-\n$1.63,0,928,$0,‐,‐ - ‐,‐,...,"715,265,270",Financial Services,VGB / DUI / JDO,$0.14,$0.1063,$1.121,6 Buy\n· 2 Hold\n· 0 Sell,Strong Buy,01/10/23,HGH
8,Value,$0.885,0.005\n(0.57%),$0.88\n-\n$0.90,"981,281","1,347,745","$871,677",$0.88,$0.88 - $0.90,$0.8885,...,"1,109,300,833",Financial Services,GMA / VGB / HGH,$0.064,$0.00,$1.297,5 Buy\n· 7 Hold\n· 0 Sell,Buy,01/10/23,JDO


In [ ]:
print(final_df.dtypes)
print(final_df.describe())

#### Export to csv file

In [79]:
# Export the DataFrame to a CSV file
final_df.to_csv('Resources/ASX_Fundamental.csv', index=False)

In [94]:
# Export the DataFrame to a CSV file
result_df.to_csv('Resources/ASX_Fundamental_Final.csv', index=False)